In [1]:
import tensorflow as tf
import numpy as np
import gym
import collections
import random
import os

In [2]:
ffmpeg_path = 'C:\\Users\\dhzns\\OneDrive\\code\\ipython\\OpenAI_gym\\ffmpeg\\bin'
os.environ['PATH'] = ffmpeg_path + ';' + os.environ['PATH']

In [3]:
#File Path
filepath_ckpt  = "./ckpt/CartPole_yt.ckpt" #weight saver check point file path
batch_size = 20
learning_rate = 0.1
input_size = 4
output_size = 2
h_size = 128
discount_rate = 0.85

In [4]:
class DQN:
    def __init__(self, session, input_size, output_size, name="main"):
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        
        self._build_network(h_size = h_size, l_rate = learning_rate)
        
    def _build_network(self, h_size=10, l_rate=0.05):
        with tf.name_scope(self.net_name) as scope:
            self._X = tf.placeholder(tf.float32, [None, input_size], name='Observation_placeholder')
            self._Y = tf.placeholder(tf.float32, [None, output_size], name='Q_placeholder')

            W1 = tf.Variable(tf.truncated_normal([self.input_size, h_size], dtype=tf.float32), trainable=True, name='weight')
            l1p = tf.matmul(self._X,W1)
            l1 = tf.nn.tanh(l1p)
        
#             tf.contrib.layers.batch_norm(l1)
            W2 = tf.Variable(tf.truncated_normal([h_size, output_size], dtype=tf.float32), trainable=True, name='weight')
            l2p = tf.matmul(l1, W2)
#             l2 = tf.nn.relu(l2p)
      
#             tf.contrib.layers.batch_norm(l2)
#             W3 = tf.Variable(tf.truncated_normal([h_size, h_size], dtype=tf.float32), trainable=True, name='weight')
#             l3p = tf.matmul(l2, W3)
#             l3 = tf.nn.relu(l3p)
        
#             tf.contrib.layers.batch_norm(l3)
#             W4 = tf.Variable(tf.truncated_normal([h_size, self.output_size], dtype=tf.float32), trainable=True, name='weight')
#             l4p = tf.matmul(l3, W4)
#             l4 = tf.nn.relu(l4p)
            
            self.logits = l2p
#             self.prob = tf.nn.tanh(self.logits)

            self._loss = tf.reduce_sum(tf.square(self._Y - self.logits))
            self._train = tf.train.AdamOptimizer(learning_rate=l_rate).minimize(self._loss)
            
    def predict(self, state):
        x = np.reshape(state, [1, self.input_size])
        return self.session.run(self.logits, feed_dict={self._X: x})
    
    def update(self, x_stack, y_stack):
        return self.session.run([self._loss, self._train], feed_dict={self._X: x_stack, self._Y: y_stack})

In [5]:
def replay_train(mainDQN, targetDQN, train_batch):
    x_stack = np.empty(0).reshape(0, input_size)
    y_stack = np.empty(0).reshape(0, output_size)
    
    for state, action, reward, next_state, done in train_batch:
        Q = mainDQN.predict(state)
        
        if done:
            Q[0, action]= reward
        else:
            Q[0, action]= reward + discount_rate * np.max(targetDQN.predict(next_state))
        
        y_stack = np.vstack([y_stack, Q])
        x_stack = np.vstack([x_stack, state])
        
    return mainDQN.update(x_stack, y_stack)

In [6]:
env = gym.make('CartPole-v0')
env = gym.wrappers.Monitor(env, './tmp/cartpole-v0-experiment2', force=True )
max_episode = 2000
replay_buffer = collections.deque()
REPLAY_MEMORY = 4000

[2017-08-30 20:57:48,526] Making new env: CartPole-v0
[2017-08-30 20:57:48,548] Clearing 20 monitor files from previous run (because force=True was provided)


In [7]:
sess = tf.Session()

In [8]:
mainDQN = DQN(sess, input_size, output_size, name="main")
targetDQN = DQN(sess, input_size, output_size, name="target")

saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())

# saver.restore(sess, filepath_ckpt)

In [9]:
def get_copy_var_ops(*, dest_scope_name="target", src_scope_name="main"):
    op_holder = []
    
    src_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=src_scope_name)
    dest_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=dest_scope_name)
    
    for src_var, dest_var in zip(src_vars, dest_vars):
        op_holder.append(dest_var.assign(src_var.value()))
        
    return op_holder

In [10]:
copy_ops = get_copy_var_ops(dest_scope_name="target", src_scope_name = "main")
sess.run(copy_ops)
print()

In [ ]:
p= 0.2
gamma = 0.8
for i_episode in range(1000):
    if p > 0:
        p -=0.01
    s = env.reset()
    for t in range(201):
#         env.render()
        e = random.random()
#         print(e)
        if e < p:
            a = env.action_space.sample()
        else:
#             print("select")
            a = np.argmax(mainDQN.predict(s))
#         print(action)
        
        #get new state    
        s1, rwd, done, _ = env.step(a)
        
        if t == 199:
            replay_buffer.append((s, a, rwd, s1, done))
            if len(replay_buffer) > REPLAY_MEMORY:
                replay_buffer.popleft()
            break
        if done:
            if t <10:
                rwd = -200
            elif t< 50:
                rwd = -150
            elif t< 100:
                rwd = -100
            elif t< 150:
                rwd = -50
        replay_buffer.append((s, a, rwd, s1, done))
        if len(replay_buffer) > REPLAY_MEMORY:
            replay_buffer.popleft()
        
        if done:
            break
            
        s = s1
        
    
            
        
    if i_episode % 2 == 0: 
        print("Episode finished after {} timesteps".format(t+1))

        for _ in range(50):
            mini_batch = random.sample(replay_buffer, batch_size)
            loss, _ = replay_train(mainDQN, targetDQN, mini_batch)
        print("Loss : ", loss)
        sess.run(copy_ops)

[2017-08-30 20:57:51,791] Starting new video recorder writing to C:\Users\dhzns\OneDrive\code\ipython\OpenAI_gym\tmp\cartpole-v0-experiment2\openaigym.video.0.12132.video000000.mp4


Episode finished after 29 timesteps


[2017-08-30 20:57:54,160] Starting new video recorder writing to C:\Users\dhzns\OneDrive\code\ipython\OpenAI_gym\tmp\cartpole-v0-experiment2\openaigym.video.0.12132.video000001.mp4


Loss :  10789.2
Episode finished after 121 timesteps
Loss :  6637.59
Episode finished after 200 timesteps
Loss :  391.818
Episode finished after 36 timesteps


[2017-08-30 20:57:58,400] Starting new video recorder writing to C:\Users\dhzns\OneDrive\code\ipython\OpenAI_gym\tmp\cartpole-v0-experiment2\openaigym.video.0.12132.video000008.mp4


...

Loss :  196.734
Episode finished after 200 timesteps
Loss :  55.5016
Episode finished after 200 timesteps
Loss :  18.9841
Episode finished after 200 timesteps
Loss :  36.0502
Episode finished after 200 timesteps
Loss :  40.0979
Episode finished after 200 timesteps
Loss :  40.5958
Episode finished after 200 timesteps
Loss :  30.5854
Episode finished after 200 timesteps
Loss :  37.7919
Episode finished after 200 timesteps
Loss :  33.6748
Episode finished after 200 timesteps
Loss :  19.8382
Episode finished after 200 timesteps
Loss :  26.6069
Episode finished after 200 timesteps
Loss :  20.4417
Episode finished after 200 timesteps
Loss :  35.8241
Episode finished after 200 timesteps
Loss :  32.6488
Episode finished after 200 timesteps
Loss :  53.0289
Episode finished after 200 timesteps
Loss :  42.7408
Episode finished after 200 timesteps
Loss :  23.3931
Episode finished after 200 timesteps
Loss :  29.565
Episode finished after 200 timesteps


[2017-08-30 20:58:12,534] Starting new video recorder writing to C:\Users\dhzns\OneDrive\code\ipython\OpenAI_gym\tmp\cartpole-v0-experiment2\openaigym.video.0.12132.video000064.mp4


Loss :  41.6347
Episode finished after 200 timesteps
Loss :  49.9567
Episode finished after 200 timesteps
Loss :  33.3239
Episode finished after 200 timesteps
Loss :  23.1768
Episode finished after 200 timesteps
Loss :  30.2106
Episode finished after 200 timesteps
Loss :  12.5499
Episode finished after 200 timesteps
Loss :  36.5756
Episode finished after 200 timesteps
Loss :  19.4979
Episode finished after 200 timesteps
Loss :  25.5268
Episode finished after 200 timesteps
Loss :  44.211
Episode finished after 200 timesteps
Loss :  30.8338
Episode finished after 200 timesteps
Loss :  44.5301
Episode finished after 200 timesteps
Loss :  34.1146
Episode finished after 200 timesteps
Loss :  39.2852
Episode finished after 200 timesteps
Loss :  22.4312
Episode finished after 200 timesteps
Loss :  29.5449
Episode finished after 200 timesteps
Loss :  37.227
Episode finished after 200 timesteps
Loss :  33.8985
Episode finished after 200 timesteps
Loss :  40.5463
Episode finished after 200 times

[2017-08-30 20:58:26,652] Starting new video recorder writing to C:\Users\dhzns\OneDrive\code\ipython\OpenAI_gym\tmp\cartpole-v0-experiment2\openaigym.video.0.12132.video000125.mp4


Loss :  50.2951


In [ ]:
saver.save(sess, filepath_ckpt)

In [ ]:
env.close()

In [ ]:
gym.upload('./tmp/cartpole-v0-experiment2', api_key='sk_R6pSeGWSRfSWyCxZP08FiQ')

In [ ]:
# for the test
env = gym.make('CartPole-v0')
for i_episode in range(20):
    s = env.reset()
    for t in range(201):
        env.render()
        a = np.argmax(mainDQN.predict(s))
        s, rwd, done, _ = env.step(a)
        
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break

In [ ]:
env.close()
sess.close()